In [ ]:
import math
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
import re

In [35]:
batch_size = 50 # b
sequence_l = 128 # n
d_model = 768 # d_model， embedding dim
num_layer = 12 # number of block stacked
number_head = 8 # multihead attention
d_ff = 2048 # feedforward dimension
block_size = 128
device = 'cuda' if torch.cuda.is_available() else 'cpu'



In [36]:
def read_data(): 
    text = open('data.txt', 'r').read()
    return re.sub('[^A-Za-z:]+',' ',text).strip().lower()

data = read_data()
print(data[:100])

first citizen: before we proceed any further hear me speak all: speak speak first citizen: you are a


In [37]:
chars = sorted(list(set(data)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [38]:
data = torch.tensor(encode(data), dtype=torch.long)
print(data)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

tensor([ 7, 10, 19,  ..., 10, 15,  8])


In [39]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [28]:
x, y = get_batch('train')


tensor([21,  0, 21,  9, 16, 22,  8,  9, 21,  0, 21,  9,  6, 14,  0, 20, 22, 19,
         6,  0, 16,  7,  0, 26, 16, 22,  0,  4, 16, 19, 10, 16, 13,  2, 15, 22,
        20,  1,  0, 21,  9, 10, 20,  0, 13,  2, 20, 21,  0, 16, 13,  5,  0, 14,
         2, 15,  0, 24,  9, 16, 14,  0, 24, 10, 21,  9,  0,  2,  0,  4, 19,  2,
         4, 12,  0,  5,  0,  9,  6,  2, 19, 21,  0, 10,  0,  9,  2, 23,  6,  0,
        20,  6, 15, 21,  0, 21, 16,  0, 19, 16, 14,  6,  0, 13, 16, 23,  6,  5,
         0, 14,  6,  0,  2,  3, 16, 23,  6,  0, 21,  9,  6,  0, 14,  6,  2, 20,
        22, 19])
tensor([ 0, 21,  9, 16, 22,  8,  9, 21,  0, 21,  9,  6, 14,  0, 20, 22, 19,  6,
         0, 16,  7,  0, 26, 16, 22,  0,  4, 16, 19, 10, 16, 13,  2, 15, 22, 20,
         1,  0, 21,  9, 10, 20,  0, 13,  2, 20, 21,  0, 16, 13,  5,  0, 14,  2,
        15,  0, 24,  9, 16, 14,  0, 24, 10, 21,  9,  0,  2,  0,  4, 19,  2,  4,
        12,  0,  5,  0,  9,  6,  2, 19, 21,  0, 10,  0,  9,  2, 23,  6,  0, 20,
         6, 15, 21,  0,

10.788929 M parameters
step 0: train loss 4.2227, val loss 4.2305
step 500: train loss 1.7467, val loss 1.9031


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 100663296 bytes.